In [24]:
import numpy as np
import pandas as pd

In [25]:
data = pd.read_csv('HousingData.csv')
display(data.head(15))

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,NaN,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,NaN,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,18.9


Aqui, podemos observar que temos um DataFrame composto por valores numéricos. Essa característica nos permite elaborar uma **Regression Tree**. Vemos que existem certos valores NaN, então vamos trata-los

In [26]:
data.isna().sum()

CRIM       20
ZN         20
INDUS      20
CHAS       20
NOX         0
RM          0
AGE        20
DIS         0
RAD         0
TAX         0
PTRATIO     0
B           0
LSTAT      20
MEDV        0
dtype: int64

Após executar o seguinte código:

```python
print(df.shape[0])         
df = df.dropna()
print(df.shape[0]) 
```

Foi possível verificar que a remoção das linhas contendo valores `NaN` resultou em uma redução de 79 linhas no conjunto de dados.  
Esta quantidade representa uma perda considerável e pode impactar negativamente a distribuição e a representatividade dos dados para as análises subsequentes.

In [27]:
print(len(data))
test = np.unique(data['ZN'])
print (len(test))
print(len(data))
test1 = np.unique(data['INDUS'])
print (len(test1))

506
27
506
77


**Observação:**  
Nas colunas `ZN` e `INDUS` , há muitos dados repetidos. Portanto, a aplicação da **moda** (valor mais frequente) para preencher valores ausentes pode trazer resultados mais confiáveis


In [28]:
data['CHAS'] = data['CHAS'].fillna(0)
data['ZN'] = data['ZN'].fillna(data['ZN'].mode()[0])
data['CRIM'] = data['CRIM'].fillna(data['CRIM'].mean())
data['AGE']  = data['AGE'].fillna(data['AGE'].mean())
data['LSTAT'] = data['LSTAT'].fillna(data['LSTAT'].mean())
data['INDUS'] = data['INDUS'].fillna(data['INDUS'].mode()[0])

In [29]:
data.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [30]:
def entropy(col):
    counts = np.unique(col, return_counts = True)
    size = len(col)
    ent = 0
    for ix in counts[1]:
        pi = ix/size
        ent += -pi*np.log2(pi)
    return ent

In [31]:
def info_gain(df, attr, target):
    valores_unicos = np.sort(np.unique(df[attr]))
    best_split = -np.inf
    best_thresh = 0
    thresholds = [(valores_unicos[i] + valores_unicos[i + 1])/2 for i in range (len(valores_unicos) - 1)]
    for thresh in thresholds:
        left = df[df[attr] <= thresh][target]
        right = df[df[attr] > thresh][target]
        ent_split = len(left)/len(data) * entropy(left) + len(right)/len(data) * entropy(right)
        gain = entropy(df[target]) - ent_split
        if gain > best_split:
            best_split = gain
            best_thresh = thresh
    return best_split, best_thresh

In [32]:
class RegressionTree:
    def __init__(self, depth = 0, max_depth = 6):
        self.attr = None
        self.children = {}
        self.threshold = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
    def train(self, df, features, target):
        if len(np.unique(df[target])) == 1 or len(features) == 1 or self.depth >= self.max_depth:
            self.target = df[target].mean()
            return
        results = [info_gain(df, attr, target) for attr in features]
        gains = [x[0] for x in results]
        best_id = np.argmax(gains)
        best_attr = features[best_id]
        best_gain, best_thresh = results[best_id]
        if best_gain == 0 or np.isnan(best_gain):
            self.target = df[target].mean()
            return

        self.attr = best_attr
        self.threshold = best_thresh
        self.children = {}
        
        left = df[df[best_attr] <= best_thresh]
        right = df[df[best_attr] > best_thresh]
        self.children['left'] = RegressionTree(self.depth + 1, max_depth=self.max_depth)
        self.children['left'].train(left, features, target)
        self.children['right'] = RegressionTree(self.depth + 1, max_depth = self.max_depth)
        self.children['right'].train(right, features, target)
    def predict (self,row):
        if self.attr is None or self.children == {}:
            return self.target
        val = row[self.attr]
        if self.threshold is not None:
            if val <= self.threshold:
                return self.children['left'].predict(row)
            else:
                return self.children['right'].predict(row)
                
        

In [33]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['MEDV'])
y = data['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = X_train.copy()
train_data['MEDV'] = y_train.values
test_data = X_test.copy()
test_data ['MEDV'] = y_test.values

In [41]:
tree = RegressionTree(max_depth=6)

x_list = list(X_train.columns)
tree.train(train_data, x_list , 'MEDV')


In [35]:
for i in range(len(test_data)):
    test_row = test_data.iloc[i]
    print(f"Exemplo {i+1}: Previsto={tree.predict(test_row)}, Real={test_row['MEDV']}")

Exemplo 1: Previsto=23.63076923076923, Real=23.6
Exemplo 2: Previsto=28.2, Real=32.4
Exemplo 3: Previsto=19.48, Real=13.6
Exemplo 4: Previsto=21.62857142857143, Real=22.8
Exemplo 5: Previsto=17.32, Real=16.1
Exemplo 6: Previsto=19.48, Real=20.0
Exemplo 7: Previsto=21.0, Real=17.8
Exemplo 8: Previsto=16.400000000000002, Real=14.0
Exemplo 9: Previsto=19.849999999999998, Real=19.6
Exemplo 10: Previsto=20.033333333333335, Real=16.8
Exemplo 11: Previsto=21.166666666666668, Real=21.5
Exemplo 12: Previsto=19.48, Real=18.9
Exemplo 13: Previsto=9.883333333333333, Real=7.0
Exemplo 14: Previsto=21.0, Real=21.2
Exemplo 15: Previsto=19.81, Real=18.5
Exemplo 16: Previsto=21.483333333333334, Real=29.8
Exemplo 17: Previsto=21.166666666666668, Real=18.8
Exemplo 18: Previsto=9.4, Real=10.2
Exemplo 19: Previsto=34.775, Real=50.0
Exemplo 20: Previsto=17.32, Real=14.1
Exemplo 21: Previsto=25.607142857142854, Real=25.2
Exemplo 22: Previsto=25.607142857142854, Real=29.1
Exemplo 23: Previsto=13.43333333333333

In [36]:
def mse(y_test, y_pred):
    return np.mean((y_test - y_pred) ** 2)
y_test = test_data['MEDV'].values
y_pred = [tree.predict(test_data.iloc[i]) for i in range (len(test_data))]
val = mse(y_test, y_pred)
rmse = np.sqrt(val)

def rform(y_test, y_pred):
    result = 1 - np.sum((y_test - y_pred)**2)/ np.sum(((y_test - y_test.mean())**2))
    return result
val_r = rform(y_test, y_pred)

                                                    
print(f"R-Squared: {val_r}")
print(f"MSE: {val}")
print(f"RMSE:{rmse}")


R-Squared: 0.724524051067766
MSE: 20.201693504880655
RMSE:4.494629406845536


## Avaliação de Overfitting do Modelo

**Profundidade 2: MSE = 32.52   | RMSE = 5.70**

**Profundidade 3: MSE = 24 ,77  | RMSE = 4.977**

**Profundidade 4: MSE = 19.73   | RMSE = 4.42**

**Profundidade 5: MSE = 18.1733 | RMSE: 4.26**

**Profundidade 6: MSE = 15.874  | RMSE: 3.98**

**profundidade 7: MSE = 10.12   | RMSE: 3.18**

Observa-se que, para a profundidade 2, o MSE é significativamente elevado, indicando que o modelo possui baixa capacidade de ajuste. Apesar de generalizar bem, a confiabilidade das previsões é limitada.

O objetivo neste contexto é encontrar uma profundidade que ofereça equilíbrio entre **generalização** e **precisão**. As profundidades 4 e 5 se destacam por apresentarem valores de MSE e RMSE mais baixos, além de não exibirem variações bruscas em relação às demais configurações, sugerindo uma boa relação entre ajuste aos dados e capacidade de generalização.

No entanto, para garantir maior confiabilidade na escolha do modelo, é fundamental analisar conjuntamente os valores de MSE e RMSE tanto para os dados de teste quanto para os dados de treino, assegurando que não haja discrepâncias significativas entre eles.

Profundidade 6
-
**Treino - MSE: 17.04, RMSE: 4.13, R²: 0.80**

**Teste  - MSE: 20.20, RMSE: 4.49, R²: 0.72**

Profundidade 5
-
**Treino - MSE: 24.26, RMSE: 4.93, R²: 0.72**

**Teste  - MSE: 20.15, RMSE: 4.49, R²: 0.73**


Profundidade 4
-
**Treino - MSE: 26.49, RMSE: 5.15, R²: 0.70**

**Teste  - MSE: 21.99, RMSE: 4.69, R²: 0.70**

Observa-se que as profundidades 4, 5 e 6 apresentam pouca variação entre os valores de MSE, RMSE e R², tanto nos dados de treino quanto nos de teste. Como buscamos maior precisão sem comprometer a capacidade de generalização do modelo, optamos pela profundidade 6. Essa configuração proporciona o melhor equilíbrio entre desempenho e generalização, garantindo a maior precisão dentre as opções analisadas, sem incorrer em overfitting.


In [37]:
display(data.describe())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.611874,10.768775,11.361304,0.067194,0.554695,6.284634,68.518519,3.795043,9.549407,408.237154,18.455534,356.674032,12.715432,22.532806
std,8.545770,23.025124,6.837487,0.250605,0.115878,0.702617,27.439466,2.105710,8.707259,168.537116,2.164946,91.294864,7.012739,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.083235,0.000000,5.190000,0.000000,0.449000,5.885500,45.925000,2.100175,4.000000,279.000000,17.400000,375.377500,7.230000,17.025000
50%,0.290250,0.000000,9.900000,0.000000,0.538000,6.208500,74.450000,3.207450,5.000000,330.000000,19.050000,391.440000,11.995000,21.200000
75%,3.611874,0.000000,18.100000,0.000000,0.624000,6.623500,93.575000,5.188425,24.000000,666.000000,20.200000,396.225000,16.570000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [42]:
tree.train(train_data, x_list, 'MEDV')
y_train_pred = [tree.predict(train_data.iloc[i]) for i in range(len(train_data))]
y_test_pred = [tree.predict(test_data.iloc[i]) for i in range(len(test_data))]

mse_train = mse(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = rform(y_train, y_train_pred)

mse_test = mse(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
r2_test = rform(y_test, y_test_pred)

print(f"Treino - MSE: {mse_train:.2f}, RMSE: {rmse_train:.2f}, R²: {r2_train:.2f}")
print(f"Teste  - MSE: {mse_test:.2f}, RMSE: {rmse_test:.2f}, R²: {r2_test:.2f}")

Treino - MSE: 17.04, RMSE: 4.13, R²: 0.80
Teste  - MSE: 20.20, RMSE: 4.49, R²: 0.72
